### Databaser - Labb 2

# Bokhandel - Applikation

In [23]:
from sqlalchemy import create_engine, MetaData, select, func, Subquery
from sqlalchemy.engine import URL
from urllib.parse import unquote

server_name   = 'localhost'
database_name = 'labb2'

connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes"
url_string        = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try:    
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)

Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=localhost;DATABASE=labb2;Trusted_Connection=yes 

Successfully connected to labb2!


In [33]:
import pandas as pd

metadata = MetaData()

metadata.reflect(bind=engine)

books_table = metadata.tables['Böcker']
authors_table = metadata.tables['Författare']
stock_table = metadata.tables['LagerSaldo']
shops_table = metadata.tables['Butiker']

def search_books_by_title(title):
    with engine.connect() as connection:
        subq = (
            select(
                func.distinct(books_table.c.ISBN13).label('distinct_ISBN13')
            )
            .where(books_table.c.Titel.ilike(f'%{title}%'))
            .alias('subq')
        )
        stmt = (
            select(
                books_table.c.ISBN13,
                books_table.c.Titel,
                books_table.c.Språk,
                books_table.c.Pris,
                books_table.c.Utgivningsdatum,
                authors_table.c.Förnamn,
                authors_table.c.Efternamn,
                func.concat(authors_table.c.Förnamn, ' ', authors_table.c.Efternamn).label('Författare'),
                stock_table.c.Antal.label('LagerAntal'),
                shops_table.c.Namn.label('Butik')
            )
            .select_from(books_table
                .join(authors_table, books_table.c.FörfattareId == authors_table.c.Id)
                .join(stock_table, books_table.c.ISBN13 == stock_table.c.ISBN13)
                .join(shops_table, stock_table.c.ButikId == shops_table.c.Id)
                .join(subq, books_table.c.ISBN13 == subq.c.distinct_ISBN13)
            )
            .where(books_table.c.Titel.ilike(f'%{title}%'))
        )
        results = connection.execute(stmt).fetchall()
        column_names = [
            'Titel', 'Pris', 'Författare', 'LagerAntal', 'Butik'
        ]
        result_dicts = [
            {column: getattr(book, column) for column in column_names}
            for book in results
        ]
        return result_dicts

def main():
    print("Welcome to the Book Search Program!")
    while True:
        print("\nOptions:")
        print("1. Search for a book by title")
        print("2. Exit")
        choice = input("Enter your choice (1/2): ")

        if choice == '1':
            title = input("Enter the book title to search: ")
            results = search_books_by_title(title)
            if results:
                df = pd.DataFrame(results)
                print("\nSearch Results:")
                display(df)
            else:
                print("\nNo books found with that title.")
            break
        elif choice == '2':
            print("Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()

Welcome to the Book Search Program!

Options:
1. Search for a book by title
2. Exit

Search Results:


,Titel,Pris,Författare,LagerAntal,Butik
0,Clean Code,40.00,Robert C. Martin,4,Fredriks gamla böcker
1,Code,35.99,Charles Petzold,7,Fredriks gamla böcker
2,Deep Learning for Coders with Fastai and PyTorch:,38.49,Jeremy Howard,4,Fredriks gamla böcker
3,Clean Code in PHP,34.99,Carsten Windler,3,Fredriks gamla böcker
4,Clean Code,40.00,Robert C. Martin,7,Everybooks
5,Code,35.99,Charles Petzold,0,Everybooks
6,Deep Learning for Coders with Fastai and PyTorch:,38.49,Jeremy Howard,7,Everybooks
7,Clean Code in PHP,34.99,Carsten Windler,8,Everybooks
8,Clean Code,40.00,Robert C. Martin,1,Johanssons script & pergament
9,Code,35.99,Charles Petzold,9,Johanssons script & pergament
